<a href="https://colab.research.google.com/github/kjedrzejczak/Wprowadzenie_do_sieci_neuronowych-Tensorflow_2.0-Keras/blob/main/28_Sie%C4%87_warstwa_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
!unzip -q reviews.zip

--2024-07-04 07:47:15--  https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42878657 (41M) [application/x-zip-compressed]
Saving to: ‘reviews.zip’

reviews.zip         100%[===================>]  40.89M  15.9MB/s    in 2.6s    

2024-07-04 07:47:18 (15.9 MB/s) - ‘reviews.zip’ saved [42878657/42878657]



# przygotowanie danych do modelu

In [3]:
"""👉Zbiór treningowy👈"""

data_dir = './reviews'
train_dir = os.path.join(data_dir, 'train')

train_texts = []  #✨
train_lables = []  #✨

for i in ['neg', 'pos']:
  dir_name = os.path.join(train_dir, i)     # iteracja pod katalogach "neg" oraz "poz" oraz tworzenie do niej scieżki
  for j in os.listdir(dir_name):            # listy plików w katalogach "neg" oraz "poz"
     if j[-4:] == '.txt':                   # sprawdzanie rozszerzenia
      f = open(os.path.join(dir_name, j))   # otwieranie pliku
      train_texts.append(f.read())          # dodawanie pliku do listy train_texts = []
      f.close()                             # zamykanie pliku
      if i == 'neg':                        # jeżeli plik był w katalogu "neg" dodawanie etykiety 0
        train_lables.append(0)              # dodawanie etykiety 0
      else:
        train_lables.append(1)              # jezlei nie dodawanie etykiety 1

In [4]:
"""👉Zbiór testowy👈"""

data_dir = './reviews'
test_dir = os.path.join(data_dir, 'test')

test_texts = []  #✨
test_lables = []  #✨

for i in ['neg', 'pos']:
  dir_name = os.path.join(test_dir, i)     # iteracja pod katalogach "neg" oraz "poz" oraz tworzenie do niej scieżki
  for j in os.listdir(dir_name):            # listy plików w katalogach "neg" oraz "poz"
     if j[-4:] == '.txt':                   # sprawdzanie rozszerzenia
      f = open(os.path.join(dir_name, j))   # otwieranie pliku
      test_texts.append(f.read())          # dodawanie pliku do listy test_texts = []
      f.close()                             # zamykanie pliku
      if i == 'neg':                        # jeżeli plik był w katalogu "neg" dodawanie etykiety 0
        test_lables.append(0)              # dodawanie etykiety 0
      else:
        test_lables.append(1)              # jezlei nie dodawanie etykiety 1

# przegląd danych

train

In [5]:
train_texts[0]

'THE RINGMASTER stars Jerry Springer as a TV talkshow host called Jerry , but it`s not THE JERRY SPRINGER SHOW , his guests are trailer trash , but not the trailer trash you get on THE JERRY SPRINGER SHOW, they attack one another , but not like on.....What is the point of making a movie about THE JERRY SPRINGER show and pretending it`s not THE JERRY SPRINGER SHOW ? And on top of that this is a very boring film'

In [6]:
train_lables[0]

0

In [7]:
print(len(train_texts))
print(len(train_lables))

25000
25000


test

In [8]:
test_texts[0]

'Although not a big Coen brothers fan, I am an admirer of their dark humor films like \'Fargo\' and \'Miller\'s Crossing.\' I have been much less impressed by their other comic mode, goofy-camp (or is that Camp Goofy?) Unfortunately, \'O Brother, Where Art Thou?\' falls into the latter category and isn\'t even as good as \'The Big Lebowski\' or \'The Hudsucker Proxy.\'<br /><br />\'O Brother\' is basically an episodic series of in-jokes without much point, and not all that much humor or cleverness either. As most reviewers have noted, the film\'s plot is very loosely \x96 and, as far as I could tell, quite arbitrarily \x96 based on The Odyssey. Its main character, Ulysses Everett McGill (George Clooney), is an \'adventurer\' like his Homeric namesake and has as his \'real\' goal the return home to prevent the marriage of his wife Penny to a suitor. Of course, we don\'t discover that this is what the Clooney/Ulysses character really wants until long after we\'ve stopped caring. And what

In [9]:
test_lables[0]

0

In [10]:
print(len(test_texts))
print(len(test_lables))

25000
25000


# Parametry Tokenazer3

In [11]:
maxlen = 100            # skracamy recenzje do 100 słów
num_words = 10000       # 10000 najczęsciej pojawiających się słów
embedding_dim = 100


tokenizer = Tokenizer(num_words=num_words)    # tworzenie instancji klasy Tokenizer --> 10000
tokenizer.fit_on_texts(train_texts)           # dopasownie tokenazer

In [12]:
list(tokenizer.index_word.items())[:20]   # sprawdznie pierwszych 20 wartości

[(1, 'the'),
 (2, 'and'),
 (3, 'a'),
 (4, 'of'),
 (5, 'to'),
 (6, 'is'),
 (7, 'br'),
 (8, 'in'),
 (9, 'it'),
 (10, 'i'),
 (11, 'this'),
 (12, 'that'),
 (13, 'was'),
 (14, 'as'),
 (15, 'for'),
 (16, 'with'),
 (17, 'movie'),
 (18, 'but'),
 (19, 'film'),
 (20, 'on')]

In [13]:
sequences = tokenizer.texts_to_sequences(train_texts)  # konwersja na sewkęcję względem tokenazera

In [14]:
for i in sequences[:3]:   # sprawdzanie trzech pierwszych sekwencji zakodowaneho tokenajzera (indeksy słów w recenzjach)
  print(i)

[1, 378, 1513, 5794, 14, 3, 245, 3210, 443, 1513, 18, 9, 587, 21, 1, 1513, 5794, 120, 24, 5514, 23, 1467, 1153, 18, 21, 1, 1467, 1153, 22, 76, 20, 1, 1513, 5794, 120, 33, 1271, 28, 157, 18, 21, 37, 20, 48, 6, 1, 210, 4, 228, 3, 17, 41, 1, 1513, 5794, 120, 2, 4449, 9, 587, 21, 1, 1513, 5794, 120, 2, 20, 347, 4, 12, 11, 6, 3, 52, 354, 19]
[2665, 26, 1478, 243, 11, 6, 2615, 4252, 1468, 2410, 90, 80, 450, 9, 735, 1, 526, 5, 815, 80, 3, 198, 4, 297, 12, 3, 695, 97, 27, 36, 109, 688, 205, 4, 5844, 5, 109, 227, 314, 238, 12, 29, 5845, 8, 1, 1773, 2, 1, 828, 1067, 6, 1051, 2, 12, 1, 159, 2471, 61, 1414, 23, 2932, 3264, 6354, 191, 967, 2, 1, 1525, 1951, 339, 12, 11, 6041, 9878, 67, 3309, 29, 3606, 6985, 2, 1028, 709, 16, 3950, 8, 3, 93, 12, 1, 2933, 467, 12, 29, 4287, 16, 1, 175, 2, 6, 3, 49, 151, 454, 12, 1, 3734, 3606, 215, 3, 1763, 2, 9259, 7916, 2, 12, 72, 141, 25, 66, 2425, 208, 16, 95, 142, 8, 1, 1351, 2, 414, 1079, 12, 1, 1918, 4, 1, 175, 59, 2718, 5, 7262, 3, 5745, 426, 3493, 7, 7, 42, 

In [15]:
word_index = tokenizer.word_index  # przypisanie do zmiennej word_index indeksu naszego tokenazera

In [16]:
print(f"{len(word_index)} ----> uniklanych słów")

88582 ----> uniklanych słów


In [17]:
# skracamy recenzje do 100 słów
maxlen = 100
train_data = pad_sequences(sequences, maxlen=maxlen)   # ograniecznie naszych tablic to 100 pierwszych wartości   ✨
train_data.shape   # sprawdzenie rozmiaru

(25000, 100)

In [18]:
train_data[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,  378, 1513, 5794,   14,    3,  245, 3210,  443,
        1513,   18,    9,  587,   21,    1, 1513, 5794,  120,   24, 5514,
          23, 1467, 1153,   18,   21,    1, 1467, 1153,   22,   76,   20,
           1, 1513, 5794,  120,   33, 1271,   28,  157,   18,   21,   37,
          20,   48,    6,    1,  210,    4,  228,    3,   17,   41,    1,
        1513, 5794,  120,    2, 4449,    9,  587,   21,    1, 1513, 5794,
         120,    2,   20,  347,    4,   12,   11,    6,    3,   52,  354,
          19],
       [   7,    7,   42, 2753,    2,    1,  120,   61, 6192,    1, 2567,
          12,    9,  119,   31,  266,    5, 2205,    8,   20,   46, 1207,
         544,    8,    1,  701,    2,  257,  314, 2766,  245, 1414, 1114,
          91,   51,    9,   13,   90,  696,   13,  128,    3, 6108, 1059,
           2, 8288, 343

In [19]:
"""przekształcenie listy do tablicy numpy"""
train_labels = np.asarray(train_lables)
train_labels

array([0, 0, 0, ..., 1, 1, 1])

In [20]:
"""przemieszanie próbek"""
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

In [21]:
"""👉podział na zbiór treningowy i walidacyjny👈"""

training_samples = 15000
validation_samples = 10000

X_train = train_data[:training_samples]
y_train = train_labels[:training_samples]
X_val = train_data[training_samples: training_samples + validation_samples]
y_val = train_labels[training_samples: training_samples + validation_samples]

# Budowa modelu
* warstwa embedding

In [22]:
maxlen = 100            # skracamy recenzje do 100 słów
num_words = 10000       # 10000 najczęsciej pojawiających się słów
embedding_dim = 100

# budowa modelu
# Embedding(input_dim, output_dim)

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1160033 (4.43 MB)
Trainable params: 1160033 (4.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
469/469 [==============================] - 21s 37ms/step - loss: 0.4871 - accuracy: 0.7551 - val_loss: 0.3806 - val_accuracy: 0.8266
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.1981 - accuracy: 0.9265 - val_loss: 0.3888 - val_accuracy: 0.8335
Epoch 3/5
469/469 [==============================] - 4s 10ms/step - loss: 0.0383 - accuracy: 0.9916 - val_loss: 0.5133 - val_accuracy: 0.8224
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0044 - accuracy: 0.9994 - val_loss: 0.6247 - val_accuracy: 0.8278
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 7.2770e-04 - accuracy: 0.9999 - val_loss: 0.7182 - val_accuracy: 0.8262


In [25]:
def plot_hist(history):
    import pandas as pd
    import plotly.graph_objects as go
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='accuracy vs. val accuracy', xaxis_title='Epoki', yaxis_title='accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='loss vs. val loss', xaxis_title='Epoki', yaxis_title='loss', yaxis_type='log')
    fig.show()

plot_hist(history)

👆 model zaczyna się przeuczać w stosunku do zbioru treningowego --> val_los rośnie, natomiast na zbiorze walidacyjnym accuracy nie rośnie

In [28]:
sequences = tokenizer.texts_to_sequences(test_texts)  # dostosowanie zbioru testowego do modelu
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_lables)

model.evaluate(X_test, y_test, verbose=0)

[0.682360053062439, 0.8266000151634216]